# Part-of-speech tagging with HMMs

In this assignment, we'll train a Hidden Markov Model (HMM) as a part-of-speech (POS) tagger, and implement both Forward-Backward inference and Viterbi decoding.

In particular:
- **(a)** Use the portion of the Penn Treebank available in the NLTK to estimate the transition and emission probabilities.
- **(b)** Implement the Forward-Backward algorithm for marginal inference of $ P(y_i\ |\ x) $
- **(c)** Implement the Viterbi algorithm to find the most likely tag _sequence_ $ \hat{y} = \arg\max_{y'} P(y'\ |\ x) $

**Note:** in the interest of a shorter assignment and giving you more time to work on projects, we've implemented **(a)** the training code and part of **(b)** the `backward()` algorithm. Do look over the solutions for those parts, as they're a good guide to the rest of your implementation!

You may want to review the Part of Speech async, as well as the slides on part-of-speech tagging which this assignment will follow closely.

## Inspect NLTK/Penn Treebank
Before continuing, let's take a few moments to inspect the format of the training data.  The Treebank Reader object has a `tagged_sents()` function that returns an list of sentences.  Each sentence consists of a list of (word, part of speech) tuples.

In [1]:
import unittest
from importlib import reload

from w266_common import utils
import pos
import pos_test
import nltk

# Use the sample of the Penn Treebank included with NLTK.
assert(nltk.download('treebank'))
corpus = nltk.corpus.treebank

print('There are %d sentences in the corpus.' % len(corpus.tagged_sents()))
print('The first sentence is:')
corpus.tagged_sents()[0]

## Part (a): HMM Parameterization

![HMM diagram](HMM.png)

Recall that a Hidden Markov Model models a sequence $[x_0, x_1, ..., x_n]$ as a Markov chain of _hidden_ states $[y_0, y_1, ..., y_n]$ and associated emissions $y_i \to x_i$ at each position. Formally, we have three sets of parameters:

1. An initial state probability $P(y_0)$ which determines the start state of the sequence.
2. Transition probabilities $P(y_i\ |\ y_{i-1})$ from one state to the next.
3. Emission probabilities $P(x_i\ |\ y_i)$ to generate an output at each timestep.

For POS tagging, we treat the word (tokens) as the observed nodes $x_i$, and the part-of-speech tags as the hidden states $y_i$ associated with each token. At training time, since the data is fully tagged, we get to observe _both_ $x_i$ and $y_i$, and so we can train our model by maximum likelihood estimation.

Recalling our n-gram models from Assignment 3, we can obtain the maximum likelihood parameters by simply counting. We'll use $t$ to denote a specific part-of-speech tag (from a tagset $T$), and $w$ to denote a specific word type (from a vocabulary $V$):

1. Initial: $ P(y_0 = t) = \frac{c(y_0 = t)}{\sum_{t' \in T} c(y_0 = t')} $
2. Transition: $ P(y_i = t\ |\ y_{i-1} = t_1) = \frac{c(t_1t)}{\sum_{t' \in T}c(t_1t')} $
3. Emission: $ P(x_i = w\ |\ y_i = t) = \frac{c(w\ |\ t)}{\sum_{w' \in V} c(w'\ |\ t)} $

### Questions
1.  Open `pos.py` and read `update_counts()` and `compute_logprobs()`.  This code should be very familiar as it's nearly identical to what you implemented in A3 to build an n-gram model.

In [2]:
reload(pos)
utils.run_tests(pos_test, ["TestTagging.test_estimate_probabilities"])

## Part (b): Forward-Backward

The forward-backward algorithm allows us to efficiently compute the most likely tag for any (or every) individual word. Formally, at each position $i$ we want to compute the marginal distribution $p(y_i\ |\ x_0, x_1, x_2, \cdots, x_n) = p(y_i\ |\ x)$. Note that taking the most likely tag from this will not necessarily find the most likely _sequence_ of tags - we'll tackle that in part(c) with Viterbi.



We found that by applying Bayes rule and decomposing (for dynamic programming speedup), $p(y_i\ |\ x)$ can be computed with the following equations. Let $n$ be the length of the sentence, and $w_i$ be the (fixed) token at position $i = 0, 1, ..., n$:

- $ \alpha(0, t) = p(t) \times p(x_0\ |\ t) $
- $ \alpha(i, t) = p(w_i\ |\ t) \times \sum_{t'} p(t\ |\ t') \times \alpha(i - 1, t') $
- $ \beta(n, t) = 1 $
- $ \beta(i-1, t) = \sum_{t'} p(w_{i}\ |\ t') \times p(t'\ |\ t) \times \beta(i, t') $

Intuitively,
- **Forward beliefs** $\alpha(i, t)$ represent the sum of all the paths that end in tag $t$ at position $i$.
- **Backward beliefs** $\beta(i, t)$ represent the sum of all the ways to continue on from tag $t$ at position $i$ through to position $n$.

If we combine the forward beliefs (information from before position $i$) with the backward beliefs (information from beyond position $i$), we get the exact probability distribution:

$$ p(y_i = t\ |\ x) = \alpha(i,t) \times \beta(i,t) $$

### Log-probabilities

Note that we're multiplying a lot of probabilities together in the above equations. While each term is easy to represent as a float, we can quickly run into numerical underflow issues when multiplying together a large number of small values.  If your dataset is as small as this and the sentences you want to tag are short (i.e. not work with real data), you can sometimes get away without worrying about this.  Alternatively, you can petition Intel to improve precision of floating point numbers close to 0.

To avoid this, we'll perform all our calculations in log space. This means that we start with the log-probabilities (as computed by `HMM.compute_logprobs()`), and replace multiplication with addition and addition with log-sum-exp, according to the identities:

- $ \log(a b) = \log(a) + \log(b) $
- $ \log(a + b) = \log(e^{\log(a)} + e^{\log(b)}) = \text{LogSumExp}(log(a), log(b)) $

To implement the latter, we recommend using [`scipy.misc.logsumexp`](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.misc.logsumexp.html), which is imported for you in the starter code.

#### Cheat Sheet:  Summing probabilities

Add probabilities together, $P(t_1) + P(t_2) + \cdots + P(t_n)$.
```python
# "Regular" Probabilities
sum_py = sum([p[t] for t in tags])
# Log-probabilities
log_sum_py = logsumexp([logp[t] for t in tags])
```

At the end of running this code,
- `sum_py` $ = \Sigma P(t_i)$
- `log_sum_py` $ = log(\Sigma P(t_i))$

Normal and log-probabilities can always be converted into each other with a $e^x$ or $log(x)$, although you shouldn't need to do this explicitly in this assignment.

_**Hint:**_ Your code in this part should look a lot like the math. In particular, `self.initial(...)`, `self.transition(...)`, and `self.emission(...)` are functions that are already set up to return appropriate defaults ($\log p(...) = \log 0 = -\infty$) for unseen tags - so you shouldn't need to check membership with `if` statements or `dict.get()`.

_**Hint:**_ There are quite a few unit tests for these functions in `pos_test.py`.  The answers there show the detais of the calculation, so you can follow along and compare to what your own code is doing.  If you're stuck, making sure you can do the examples in there on a scrap piece of paper is a great place to start.

### Questions
1.  Implement `forward()` (i.e. compute $\alpha$) in `pos.py`
2.  Implement `backward()` (i.e. compute $\beta$) in `pos.py`
3.  Inspect the code for `forward_backward()` in `pos.py`.

In [3]:
reload(pos)
utils.run_tests(pos_test, ["TestTagging.test_alpha"])

In [4]:
reload(pos)
utils.run_tests(pos_test, ["TestTagging.test_beta"])

In [5]:
reload(pos)
utils.run_tests(pos_test, ["TestTagging.test_forward_backward"])

## Let's give it a try!

(Warning: if you decide to try some of your own - and you should! - you may find the limited vocabulary of the training set to be problematic.)

In [6]:
reload(pos)
hmm = pos.HMM()
for sentence in corpus.tagged_sents():
    hmm.update_counts(sentence)
hmm.compute_logprobs()
def pretty_print_fb(sentence):
    print(sentence)
    print(hmm.forward_backward(sentence.split()))

In [7]:
pretty_print_fb('Pierre will join the board .')
pretty_print_fb('Pierre joined an organization .')
pretty_print_fb('The old man .')

## Part (c): Viterbi

Viterbi finds the maximum likelihood sequence of assignments, rather than considering a single assignment at a time.  Its implementation is a small tweak on the $\alpha$ of Forward Backward.  In particular, instead of trying to find the _sum_ of all the possible ways to make a part of speech in a particular position, we try to find the _best_ way. Formally, we have:

- $\delta(0, t) = p(t) \times P(x_0\ |\ t)$
- $\delta(i, t) = p(x_i\ |\ t) \times \max_{t'} \left[\delta(i - 1, t') \times p(t\ |\ t')\right]$

_**Hint:**_ As in part (b), your code should look quite a lot like the math above.

### Questions:
1.  Read the `viterbi()` function at the bottom of `pos.py`.  It uses the `delta` table and backpointers to determine the most likely sequence of part of speech tags.
2.  Implement the equations immediately above pos.py's `build_viterbi_delta()`.
3.  The derivation of forward/backward and viterbi both start with the computation of the probability of the joint probability of the states and emissions.  How do they differ?
4.  Is it possible that the probability of the joint assignment with the same state values as the most probable forward/backward solution for each position is 0?

In [8]:
reload(pos)
utils.run_tests(pos_test, ["TestTagging.test_build_viterbi_delta"])

In [9]:
reload(pos)
utils.run_tests(pos_test, ["TestTagging.test_viterbi_end_to_end"])

## Let's give it a try!

(Warning: if you decide to try some of your own - and you should! - you may find the limited vocabulary of the training set to be problematic.)

In [10]:
corpus = nltk.corpus.treebank
# Uncomment below if you install the full Penn Treebank
# corpus = nltk.corpus.ptb

# Optional: set to true to get a nice progressbar during training.
use_fancy_progressbar = True
if use_fancy_progressbar:
    utils.require_package("tqdm")
    from tqdm import tqdm as ProgressBar
else:
    ProgressBar = lambda x: x  # pass-through iterator

reload(pos)
hmm = pos.HMM()
for sentence in ProgressBar(corpus.tagged_sents()):
    hmm.update_counts(sentence)
hmm.compute_logprobs()

In [11]:
def pretty_print_v(sentence):
    tokens = sentence.split()
    tags = hmm.viterbi(tokens)
    print(" ".join("%s/%s" % (w,t) for (w,t) in zip(tokens, tags)))

pretty_print_v('Pierre will join the board .')
pretty_print_v('Pierre joined an organization .')
pretty_print_v('The old man .')